In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'  # default is ‘last_expr'

%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/home/mink/notebooks/CameraTraps')  # append this repo to PYTHONPATH
sys.path.append('/home/mink/lib/ai4eutils')

In [3]:
import json
import os
from collections import Counter, defaultdict
from random import sample
from shutil import copyfile
from multiprocessing.pool import ThreadPool

from tqdm import tqdm
from unidecode import unidecode 

from data_management.megadb.schema import sequences_schema_check
from data_management.megadb.converters.cct_to_megadb import make_cct_embedded, process_sequences, write_json

# wellington_nz

This notebook is a template for how new datasets can be formatted for ingestion into the database.

The ideal dataset has both **location** and **sequence** information, in addition to any species or bounding box labels.

Give the path to a JSON file where output from this script will be written to. You can then take this file to the .Net app for ingestion to the database.

In [4]:
dataset_name = 'wellington_nz'

container_root = '/mink_disk_0/camtraps/wellington_nz'  # AzCopied the container to data disk
path_prefix = 'images'

path_to_output = f'/home/mink/camtraps/data/megadb_jsons/{dataset_name}.json' 
path_to_output_temp = f'/home/mink/camtraps/data/megadb_jsons/{dataset_name}_temp.json' 

## Step 0 - Add an entry to the `datasets` table

Done

## Step 1 - Prepare the `sequence` objects to insert into the database

### Step 1a - If you have metadata in COCO Camera Traps (CCT) format already...

For a dataset, you probably have one or two JSONs in the CCT format, one containing image-level species labels and another containing bounding box annotations. Here we combine them and embed any annotation items into the image items.

In [5]:
# path to the CCT json, or a loaded json object
path_to_image_cct = '/home/mink/camtraps/data/prev_labels/wellington_camera_traps.json'  # set to None if not available
path_to_bbox_cct = None # set to None if not available
assert not (path_to_image_cct is None and path_to_bbox_cct is None)

You might like to process the resulting `embedded` dataset a little more:

- For image entries that do not have species label but have bounding box annotations, you can add a `species` field to the `annotations` field of each item in the list `embedded`, according to the `category` field of the first `bbox` item in `annotations`:
    - If `e['annotations']['bbox'][category']` is `person`, assign `['human']` to `e['annotations']['species']`. Note that it needs to be a list (of one item).
    - If `animal`, assign `['unidentified']`

In [6]:
%%time

embedded = make_cct_embedded(image_db=path_to_image_cct, bbox_db=path_to_bbox_cct)

Loading image DB...
Number of items from the image DB: 270450
Number of images with more than 1 species: 0 (0.0% of image DB)
No bbox DB provided.
CPU times: user 2.38 s, sys: 455 ms, total: 2.83 s
Wall time: 3.1 s


In the following step, properties will be moved to the highest level that is still correct, i.e. if a property at the image-level always has the smae value for all images in a sequence, it will be moved to be a sequence-level property.

If a sequence-level property has the same value throughout this dataset (often 'rights holder'), it will be removed from the `sequence` objects. A message about this will be printed, and you should add that property and its (constant) value to this dataset's entry in the `datasets` table.

In [7]:
%%time

sequences = process_sequences(embedded, dataset_name)

The dataset_name is set to wellington_nz. Please make sure this is correct!
Making a deep copy of docs...


 15%|█▌        | 41468/270450 [00:00<00:00, 414594.40it/s]

Putting 270450 images into sequences...


100%|██████████| 270450/270450 [00:00<00:00, 527698.43it/s]


Number of sequences: 90478
Checking the location field...
Checking which fields in a CCT image entry are sequence-level...

all_img_properties
{'class', 'file', 'site', 'frame_num', 'camera', 'id', 'datetime'}

img_level_properties
{'id', 'frame_num', 'file'}

image-level properties that really should be sequence-level
{'class', 'camera', 'site', 'datetime'}

Finished processing sequences.
Example sequence items:

{"dataset": "wellington_nz", "seq_id": "2", "images": [{"id": "290716114012001a1116", "frame_num": 0, "file": "290716114012001a1116.JPG"}, {"id": "290716114014001a1114", "frame_num": 1, "file": "290716114014001a1114.JPG"}, {"id": "290716114014001a1115", "frame_num": 2, "file": "290716114014001a1115.JPG"}], "class": ["bird"], "datetime": "7/29/2016 11:40", "camera": "111", "site": "001a"}

{"dataset": "wellington_nz", "seq_id": "40003", "images": [{"id": "030716222820038as242", "frame_num": 0, "file": "030716222820038as242.JPG"}, {"id": "030716222820038as243", "frame_num": 1, 

In [11]:
# only addition step: change the 'site' attribute to 'location' to be consistent with other datasets
locations = set()
for seq in sequences:
    seq['location'] = seq['site']
    del seq['site']
    
    locations.add(seq['location'])

len(locations)

215

In [13]:
# sample some sequences to make sure they are what you expect
sample(sequences, 2)

[OrderedDict([('dataset', 'wellington_nz'),
              ('seq_id', '36962'),
              ('images',
               [{'id': '190116070904022c5351',
                 'frame_num': 0,
                 'file': '190116070904022c5351.JPG'},
                {'id': '190116070904022c5352',
                 'frame_num': 1,
                 'file': '190116070904022c5352.JPG'},
                {'id': '190116070906022c5353',
                 'frame_num': 2,
                 'file': '190116070906022c5353.JPG'}]),
              ('class', ['bird']),
              ('datetime', '1/19/2016 7:09'),
              ('camera', '535'),
              ('location', '022c')]),
 OrderedDict([('dataset', 'wellington_nz'),
              ('seq_id', '50836'),
              ('images',
               [{'id': '050116122908030bs101',
                 'frame_num': 0,
                 'file': '050116122908030bs101.JPG'},
                {'id': '050116122908030bs102',
                 'frame_num': 1,
                 'file

## Step 2 - Pass the schema check

Once your metadata are in the MegaDB format for `sequence` items, we check that they conform to the format's schema.

If the format conforms, the following messages will be printed:

```
Verified that the sequence items meet requirements not captured by the schema.
Verified that the sequence items conform to the schema.
```

For large datasets, the second step will take some time (~ a minute). 

Otherwise there will be an error message describing what's wrong. Please fix the issues until all checks are passed. You might need to write some snippets of code to loop through the `sequence` items to understand which entries have problems.

In [15]:
%%time

sequences_schema_check.sequences_schema_check(sequences)

Verified that the sequence items meet requirements not captured by the schema.
Verified that the sequence items conform to the schema.
CPU times: user 14 s, sys: 379 ms, total: 14.4 s
Wall time: 14.4 s


In [16]:
with open(path_to_output_temp, 'w', encoding='utf-8') as f:
    json.dump(sequences, f, indent=1, ensure_ascii=False)

### Step 2b - copy images to flat folder

In [5]:
with open(path_to_output_temp) as f:
    sequences = json.load(f)

In [6]:
len(sequences)

90478

In [18]:
def copy_file(src_path, dst_path):
    return copyfile(src_path, dst_path)

In [13]:
%%time

path_pairs = []
for seq in tqdm(sequences):
    seq_id = seq['seq_id']
    for im in seq['images']:
        src_path = os.path.join(container_root, path_prefix, im['file'])
        # assert os.path.exists(src_path), src_path
        frame = im['frame_num']
        dst_path = os.path.join('/mink_disk_0/camtraps/imerit12', 
                                f'{dataset_name}.seq{seq_id}.frame{frame}.jpg')
        path_pairs.append((src_path, dst_path))

100%|██████████| 90478/90478 [00:01<00:00, 80242.79it/s]

CPU times: user 1.14 s, sys: 18.6 ms, total: 1.16 s
Wall time: 1.15 s


In [14]:
path_pairs[1000]

('/mink_disk_0/camtraps/wellington_nz/images/290116054720002c8232.JPG',
 '/mink_disk_0/camtraps/imerit12/wellington_nz.seq551.frame1.jpg')

In [19]:
%%time

with ThreadPool(4) as pool:
    dst_paths = pool.starmap(copy_file, path_pairs)

CPU times: user 1min 32s, sys: 5min 35s, total: 7min 8s
Wall time: 36min 25s


In [20]:
len(dst_paths)

270450

Moved the images to `imerti12a`

In [8]:
# Oops, delete images that are labeled empty

empty_images = []

for seq in tqdm(sequences):
    seq_id = seq['seq_id']
    if len(seq['class']) == 1 and seq['class'][0] == 'empty':
        for im in seq['images']:
            frame = im['frame_num']
            dst_path = os.path.join('/mink_disk_0/camtraps/imerit12a', 
                                f'{dataset_name}.seq{seq_id}.frame{frame}.jpg')
            empty_images.append(dst_path)

100%|██████████| 90478/90478 [00:00<00:00, 486512.92it/s]


In [10]:
len(empty_images)  # About 17% are empty
empty_images[100]
empty_images[-1]

45721

'/mink_disk_0/camtraps/imerit12a/wellington_nz.seq236.frame1.jpg'

'/mink_disk_0/camtraps/imerit12a/wellington_nz.seq89955.frame2.jpg'

In [12]:
def remove_file(p):
    os.remove(p)

In [14]:
%%time

with ThreadPool(8) as pool:
    dst_paths = pool.map(remove_file, empty_images)

CPU times: user 666 ms, sys: 5.66 s, total: 6.32 s
Wall time: 17.2 s
